In [ ]:
from instagram_private_api import Client, ClientCompatPatch, ClientCookieExpiredError
from instagram_private_api_extensions import pagination

In [ ]:
from datetime import datetime
from uuid import uuid4
import codecs
import time
import json
import random
from os import environ

I don't understand the api, and since I don't trust it, I've used dummy credentials set in my environment

In [ ]:
def onlogin_callback(api):
    cache_settings = api.settings
    with open(new_settings_file, 'w') as outfile:
        json.dump(cache_settings, outfile, default=to_json)
        print('SAVED: {0!s}'.format(new_settings_file))

In [ ]:
username = environ.get("TEMP_INSTAGRAM_USERNAME")
password = environ.get("TEMP_INSTAGRAM_PASSWORD")

In [ ]:
def to_json(python_object):
    if isinstance(python_object, bytes):
        return {'__class__': 'bytes',
                '__value__': codecs.encode(python_object, 'base64').decode()}
    raise TypeError(repr(python_object) + ' is not JSON serializable')

def from_json(json_object):
    if '__class__' in json_object and json_object['__class__'] == 'bytes':
        return codecs.decode(json_object['__value__'].encode(), 'base64')
    return json_object


In [ ]:
try:
    with open("cached_settings.json") as json_file:
        cached_settings = json.load(json_file, object_hook=from_json)
    api = Client(username, password, settings=cached_settings)
    cookie_expiry = api.cookie_jar.auth_expires
    print('Cookie Expiry: {0!s}'.format(datetime.fromtimestamp(cookie_expiry).strftime('%Y-%m-%dT%H:%M:%SZ')))
except ClientCookieExpiredError:
    api = Client(username, password)
    with open("cached_settings.json", "w") as json_file:
        json_file.write(json.dumps(api.settings, indent=4, default=to_json))

This is about testing that the authentication is working before we start

In [ ]:
results = api.feed_timeline()

In [ ]:
results

In [ ]:
def get_user_latest(user_id, the_username):
    rank_token = str(uuid4())
    users = []
    response = api.user_following(user_id=user_id, rank_token=rank_token)
    new_users = response["users"]
    users.extend(new_users)
    next_max_id = response["next_max_id"]
    while next_max_id:
        print(f"{the_username} - {next_max_id}")
        wait_time = random.randrange(500,1000) / 10
        time.sleep(wait_time)
        response = api.user_following(user_id=user_id, rank_token=rank_token, max_id=next_max_id)
        new_users = response["users"]
        users.extend(new_users)
        next_max_id = response.get("next_max_id")
    users.sort(key=lambda x: x.get("username"))
    with open(f"potential_savings/{user_id}_raw.json", "w") as savings_file:
        savings_file.write(json.dumps(users, indent=4))
    usernames = [user["username"] for user in users]
    with open(f"potential_savings/{user_id}_usernames.txt", "w") as usernames_file:
        usernames_file.write("\n".join(usernames))
    with open(f"potential_savings/{user_id}_id_to_username.txt", "w") as id_to_username_file:
        id_to_username_file.write(str(list(the_username)))

In [ ]:
users = [
    {"user_id": "52200311661", "the_username": "chopping.block2024"},
    {"user_id": "66383374426", "the_username": "blockout2024.02"},
]
for user in users:
    wait_time = random.randrange(500,1000) / 10
    get_user_latest(**user)